# Setup

In [1]:
# https://www.geeksforgeeks.org/how-to-run-cuda-c-c-on-jupyter-notebook-in-google-colaboratory/

In [8]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2023 NVIDIA Corporation
Built on Tue_Aug_15_22:02:13_PDT_2023
Cuda compilation tools, release 12.2, V12.2.140
Build cuda_12.2.r12.2/compiler.33191640_0


In [9]:
!pip install git+https://github.com/andreinechaev/nvcc4jupyter.git

  Cloning https://github.com/andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-tkwu5zm5
  Running command git clone --filter=blob:none --quiet https://github.com/andreinechaev/nvcc4jupyter.git /tmp/pip-req-build-tkwu5zm5
  Resolved https://github.com/andreinechaev/nvcc4jupyter.git to commit 5741c522547756ac4bb7a16df32106a15efb8a57
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for nvcc4jupyter: filename=nvcc4jupyter-1.2.1-py3-none-any.whl size=10739 sha256=99634096e9ae99b658cd96f87c6e8ccf9cbc16bf9f7ab36b8a182687000c7e83
  Stored in directory: /tmp/pip-ephem-wheel-cache-7x2yrz3b/wheels/a8/b9/18/23f8ef71ceb0f63297dd1903aedd067e6243a68ea756d6feea
Successfully built nvcc4jupyter


In [10]:
%load_ext nvcc4jupyter

Detected platform "Colab". Running its setup...
Source files will be saved in "/tmp/tmpbcr9faoc".


# Kode CUDA

In [11]:
%%writefile cuda.cu

#include <stdio.h>
#include <math.h>
#include <stdbool.h>
#define ROW_OP_THREAD 256


double *allocate_matrix(int n, bool isAugmented)
{
  int col = isAugmented ? 2*n : n;
    double *mat = (double *)malloc(n * col * sizeof(double));

    if (mat == NULL)
    {
        printf("Memory allocation failed!");
        free(mat);
        return NULL;
    }

    return mat;
}

int get_matrix_index(int row, int col, int width)
{
    return width * row + col;
}

void read_matrix(double *matrix, int n)
{
    double d = 0.0;
        for (int i = 0; i < n; i++)
        {
            for (int j = 0; j < n; j++)
            {
                scanf("%lf", &d);
                matrix[get_matrix_index(i, j, 2 * (n))] = d;
            }
        }
        for (int i = 0; i < n; ++i)
        {
            for (int j = n; j < 2 * (n); ++j)
            {
                if (j == (i + (n)))
                {
                    matrix[get_matrix_index(i, j, 2 * (n))] = 1;
                }
                else
                {
                    matrix[get_matrix_index(i, j, 2 * (n))] = 0;
                }
            }
        }
}

void print_result(double *mat, int rows)
{
    printf("%d\n", rows);
    for (int i = 0; i < rows; i++)
    {
        for (int j = rows; j < rows*2; j++)
        {
            printf("%lf ", mat[get_matrix_index(i, j, rows*2)]);
        }
        printf("\n");
    }
}

__device__ int GetMatrixIdx(int row, int col, int width)
{
    return width * row + col;
}

__global__ void NormalizePivotRow(double* mat, int n, int pivot_idx, double d) {
  // called for pivot row only
  __shared__ double lmat[ROW_OP_THREAD];
  int col = threadIdx.x + blockIdx.x * blockDim.x;

  if (col < 2 * n) {
    lmat[threadIdx.x] = mat[GetMatrixIdx(pivot_idx, col, 2*n)] / d;
    __syncthreads();
    mat[GetMatrixIdx(pivot_idx, col, 2*n)] = lmat[threadIdx.x];
  }
}

__global__ void NormalizePivotHelper(double* mat, int n, int pivot_idx) {
  double d = mat[GetMatrixIdx(pivot_idx, pivot_idx, 2*n)];
  NormalizePivotRow<<<ceil(2.0 * n / double(ROW_OP_THREAD)), ROW_OP_THREAD>>>(mat, n, pivot_idx, d);
  __syncthreads();
}

__global__ void SubtractNonPivot(double* mat, int n, int row_idx, int pivot_idx, double d) {
  // called for nonpivot only

  __shared__ double lmat[ROW_OP_THREAD];
  int col = threadIdx.x + blockIdx.x * blockDim.x;

  if (col < 2 * n) {
    lmat[threadIdx.x] = mat[GetMatrixIdx(row_idx, col, 2*n)] - (d * mat[GetMatrixIdx(pivot_idx, col, 2*n)]);
    __syncthreads();
    mat[GetMatrixIdx(row_idx, col, 2*n)] = lmat[threadIdx.x];
  }
}


__global__ void SubsNonPivotKernel(double* mat, int n, int pivot_idx, int block_size) {
  int row_size = n /block_size;
  int start_row = (threadIdx.x * row_size);
  int end_row = start_row + row_size;

  for (int row = start_row; row < end_row; row++) {
    if (row != pivot_idx) {
      double d = mat[GetMatrixIdx(row, pivot_idx, 2*n)] / mat[GetMatrixIdx(pivot_idx, pivot_idx, 2*n)];
      SubtractNonPivot<<<ceil(2.0 * n / double(ROW_OP_THREAD)), ROW_OP_THREAD>>>(mat, n, row, pivot_idx, d);
    }
  }
}

void invert_matrix(int n, double* mat) {
  double* d_mat;
  size_t size = n * n * 2 * sizeof(double);
  cudaMalloc((void**)&d_mat, size);
  cudaMemcpy(d_mat, mat, size, cudaMemcpyHostToDevice);

  int block_size = n >= 1024 ? 1024 : n;
  dim3 dimBlock(block_size);
  dim3 dimGrid(1, 1);

  for (int i=0; i<n; i++) {
    NormalizePivotHelper<<<1, 1>>>(d_mat, n, i);
    cudaDeviceSynchronize();

    SubsNonPivotKernel<<<dimGrid, dimBlock>>>(d_mat, n, i, block_size);
    cudaDeviceSynchronize();
  }

  cudaMemcpy(mat, d_mat, size, cudaMemcpyDeviceToHost);

  cudaFree(d_mat);
}

int main(void) {
  int n;
  scanf("%d", &n);

  double* mat = allocate_matrix(n, true);
  read_matrix(mat, n);

  invert_matrix(n, mat);

  print_result(mat, n);

  return 0;
}

Writing cuda.cu


Enable Relocatable Device Code (RDC) and lcudadevrt linker to enable dynamic parallelism

In [12]:
!nvcc cuda.cu -rdc=true -lcudadevrt -o cuda

# Get Testcase

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


access test case using this path /content/drive/Shareddrives/Sister/test_cases/32.txt

# Execution Cuda

Example input via stdin

In [ ]:
!time ./cuda

3 3 0 2 0 1 1 2 0 -2
0 0 2560 0 2560 0 2563
0.200000 0.000000 0.200000 
-0.200000 1.000000 0.300000 
0.200000 -0.000000 -0.300000 

real	0m2.905s
user	0m0.080s
sys	0m0.215s


In [13]:
!time ./cuda < /content/drive/Shareddrives/Sister/test_cases/32.txt > outcuda32.txt


real	0m0.478s
user	0m0.117s
sys	0m0.217s


In [14]:
!time ./cuda < /content/drive/Shareddrives/Sister/test_cases/64.txt > outcuda64.txt


real	0m0.305s
user	0m0.080s
sys	0m0.215s


In [15]:
!time ./cuda < /content/drive/Shareddrives/Sister/test_cases/128.txt > outcuda128.txt


real	0m0.443s
user	0m0.208s
sys	0m0.217s


In [16]:
!time ./cuda < /content/drive/Shareddrives/Sister/test_cases/256.txt > outcuda256.txt


real	0m0.722s
user	0m0.506s
sys	0m0.204s


In [17]:
!time ./cuda < /content/drive/Shareddrives/Sister/test_cases/512.txt > outcuda512.txt


real	0m2.502s
user	0m1.589s
sys	0m0.234s


In [18]:
!time ./cuda < /content/drive/Shareddrives/Sister/test_cases/1024.txt > outcuda1024.txt


real	0m7.911s
user	0m6.138s
sys	0m0.269s


In [19]:
!time ./cuda < /content/drive/Shareddrives/Sister/test_cases/2048.txt > outcuda2048.txt


real	0m24.686s
user	0m21.434s
sys	0m0.370s


# Execution Serial

In [3]:
!g++ /content/drive/Shareddrives/Sister/serial/serial.cpp -o serial

In [4]:
!time ./serial < /content/drive/Shareddrives/Sister/test_cases/32.txt > out32.txt


real	0m0.357s
user	0m0.003s
sys	0m0.002s


In [5]:
!time ./serial < /content/drive/Shareddrives/Sister/test_cases/64.txt > out64.txt


real	0m0.529s
user	0m0.009s
sys	0m0.003s


In [6]:
!time ./serial < /content/drive/Shareddrives/Sister/test_cases/128.txt > out128.txt


real	0m0.615s
user	0m0.089s
sys	0m0.002s


In [7]:
!time ./serial < /content/drive/Shareddrives/Sister/test_cases/256.txt > out256.txt


real	0m1.279s
user	0m0.352s
sys	0m0.004s


In [20]:
!time ./serial < /content/drive/Shareddrives/Sister/test_cases/512.txt > out512.txt


real	0m2.335s
user	0m2.307s
sys	0m0.019s


In [21]:
!time ./serial < /content/drive/Shareddrives/Sister/test_cases/1024.txt > out1024.txt


real	0m20.865s
user	0m20.586s
sys	0m0.057s


In [22]:
!time ./serial < /content/drive/Shareddrives/Sister/test_cases/2048.txt > out2048.txt


real	2m34.759s
user	2m33.350s
sys	0m0.265s
